In [17]:
import torch
from torchvision import models

# Загружаем предварительно обученную модель ResNet34
model = models.resnet34(pretrained=False)

# Загружаем веса из файла .pth
state_dict = torch.load('resnet34.pth')
model.load_state_dict(state_dict)
model.eval()

c:\Users\VIKTOR\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\VIKTOR\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [18]:
# Создание нового слоя с   1001 выходами
new_fc = torch.nn.Linear(512, 2)

# Замена старого слоя на новый
model.fc = new_fc

# Заморозка нижних слоев модели
for param in model.parameters():
    param.requires_grad = False

# Размораживание верхних слоев для обучения
for param in model.fc.parameters():
    param.requires_grad = True

In [19]:
import os
from PIL import Image
from torch.utils.data import Dataset

class MountainDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []

        # Проходим по всем файлам в каталоге
        for folder in os.listdir(root_dir):
            folder_path = os.path.join(root_dir, folder)
            if os.path.isdir(folder_path):
                # Если это директория, предполагаем, что это класс
                label = 1 if folder == 'mountain' else 0
                # Загружаем все изображения из этой директории
                for filename in os.listdir(folder_path):
                    img_path = os.path.join(folder_path, filename)
                    self.images.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)


In [20]:
from torchvision import transforms
from torch.utils.data import DataLoader

# Определение преобразований для изображений
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Создание экземпляра нашего набора данных
train_dataset = MountainDataset("seg_train", transform=transform)

# Создание DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [21]:
# Создание экземпляра нашего набора данных для теста
test_dataset = MountainDataset("seg_test", transform=transform)

# Создание DataLoader для тестовой выборки
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [22]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [23]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Предполагается, что model, criterion, train_loader, test_loader уже определены
num_epochs = 10

for epoch in range(num_epochs):
    # Обучение
    train_loss = 0
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    # Вычисление функции потерь на тестовом наборе данных
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
    test_loss /= len(test_loader)
    train_loss /= len(train_loader)
    print(f"Epoch {epoch+1}, Test Loss: {test_loss:.4f}, Train Loss: {train_loss:.4f}")


Epoch 1, Test Loss: 0.1663, Train Loss: 0.2160
Epoch 2, Test Loss: 0.1537, Train Loss: 0.1743
Epoch 3, Test Loss: 0.1536, Train Loss: 0.1665


KeyboardInterrupt: 